**Построение прогноза выживших на Титанике**


Первая попытка поучаствовать в соревнованиях Kaggle :)

**Целями этой работы являются:**
1. Тренировка воркфлоу работы с данными
2. Знакомство с функционалом Kaggle и принципами работы соревнований
3. Знакомство с ML моделями прогнозирования и особенностями подготовки данных для них

**Краткое описание проблемы** - зная из датасета **train** пассажиров, их характеристики и погибли они или нет, создать модель, которая бы предсказывала выжил или погиб пассажир в датасете **test**.

[Ссылка на детальное описание соревнования](https://www.kaggle.com/c/titanic).

Итак, приступим)


# 1. Импортируем библиотеки необходимые для первичного изучения данных

In [ ]:
#Библиотеки для работы с данными
import pandas as pd
import numpy as np

#Библиотеки для визуализации
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

Считываем датасеты, смотрим как они выглядят и сразу же проверяем описательные статистики train.\
Test пока не смотрим, т.к. там по идее тоже самое что и в train, но без указания выжил человек или погиб.

In [ ]:
test = pd.read_csv('../input/titanic/test.csv')
test.head()

In [ ]:
train = pd.read_csv('../input/titanic/train.csv')
train.head()

In [ ]:
train.describe(include="all").round()

**Первоначальные наблюдения:**
* Как и было указано в описании, всего в датасете train есть информация о 891 пассажире
* Видим что в колонках Age, Cabin и Embarked не все данные (count для этих строк меньше 891). Проверяем дополнительно с помощью **isna()**

In [ ]:
train.isna().sum()

* У нас действительно нет данных о возрасте для 177 пассажиров. Это надо будет как-то заполнить, так как возраст совершенно точно существенно влияет на выживаемость, учитывая приказ капитана Титаника о загрузке шлюпок в первую очередь женщинами и детьми (спасибо Wiki). Удалять эти строки я бы не стал, так как они составляют достаточно крупную часть датасета (около 20%).
* Также отсутствует информация о номере кабины / каюты для 687 пассажиров. Почитав форумы Keggle, нашел теорию что номера кают отсутсвуют у более бедных классов (Ticket class). Ниже построим график, чтобы это проверить и потом решим что делать.
* Отсуствие данных о порте погрузки для 2-х человек несущественно. Думаю что заменим на какое-нибудь значение.


# 2. Заполнение пропущенных значений
**Начнем с колонки age, в которой не хватает 177 значений. Наиболее очевидные варианты заполнить их это:**
1. Попробовать предсказать примерную возрастную группу с помощью гоноративов (Mr, Mrs, Miss и т.п.)
2. Взять данные с какого-нибудь сайта, где они уже есть в полном виде (ну или более полном)


Второй вариант хоть и не очень спортивен, правилами совернования не запрещен, поэтому попробуем реализовать именно его. Так мы получим наиболее точные данные о возрасте и одновременно потренируем веб-скрейпинг. Доставать данные о возрасте было решено с https://en.wikipedia.org/wiki/Passengers_of_the_Titanic.

Поскольку Kaggle не поддерживает BeautifulSoup, который я использовал для получения веб-данных, то вот [ссылка на код с github](https://nbviewer.jupyter.org/github/NikitaVhr/Training-public-/blob/2a78fa39e9336f073d4364c16c6921e26c923210/Code%20for%20Wiki%20Scraping.ipynb), итогом которого стал датасет **scraped_age**. Однако просто взять и сджойнить его здесь с данными в соревновании пока нельзя по той причине, что ключ (имя), по которому я планировал их объединить, у многих людей отличаются. Так, например, человек с именем ***Moran, Mr. James*** в одном датафрейме указан как ***Moran, Mr. Daniel James*** в данных с Wikipedia.

Решить эту проблему помог Excel, а точнее функция vlookup, в которой есть параметр, позволяющий подтянуть как точно совпадающие значения, так и приблизительные.
В итоге, перед загрузкой сюда, **scraped_age** был доработан в Excel следующим образом:
1. Объединены имена и возраст из таблиц train и test, чтобы получить единую полную таблицу для всех пассажиров
2. Сначала к именам без возраста подтянуты ВПР'ом точно совпадающие значения с Wiki
3. Затем к осташимся именам без возраста подтянуты приблизительные значения c Wiki

После описанных выше преобразований получился датасет **scraped_age** в котором для 1308 пассажиров был указан возраст.

In [ ]:
# Cчитываем датасет и смотрим как он выгялдит
scraped_age = pd.read_csv('../input/scraped-age/scraped_age.csv')
scraped_age.tail(3)

In [ ]:
# Добавляем к train данные о возрасте, которые взяли из Википедии
train = train.merge(scraped_age, on='Name', how='left')

# Заполяем пропущенные значения Age данными из датасета scraped_age
train['Age'] = train['Age'].fillna(train['Age_full'])

# Убираем только что добавленную колонку, т.к. она нам теперь не нужна
train = train.drop(['Age_full'], axis = 1)

Посколкьку у нас есть еще датасет test, сразу же смотрим его и делаем то же самое что и для train.

In [ ]:
test.isna().sum()

In [ ]:
# Добавляем данные о возрасте, которые взяли в Википедии
test = test.merge(scraped_age, on='Name', how='left')

# Заполяем пропущенные значения Age данными из датасета scraped_age
test['Age'] = test['Age'].fillna(test['Age_full'])

# Убираем только что добавленную колонку, т.к. она нам теперь не нужна и возникшие при джойне дубликаты
test = test.drop(['Age_full'], axis = 1)
test = test.drop_duplicates()

**Решаем что делать с отсутствующей информацией о номере кабины / каюты пассажиров**


Информации нет для 687 пассажиров (77% датафрейма). Это много.

Если теория о том, что номера кают отсутсвуют у более бедных классов (Ticket class), то думаю что мы эту колонку просто удалим, так как предполагаю, что номер каюты должен быть тесно связан с классом, информация о котором у нас есть в полном объеме.

In [ ]:
# Задаем параметры для отображения нескольких графиков + настраиваем их размер
fig, ax = plt.subplots(1, 2,figsize=(15,5))

# Создаем "булевую" колонку, чтобы посмотреть влияет ли наличие информации о каюте как-то на выживаемость
train["CabinBool"] = train["Cabin"].notnull().astype('int')
test["CabinBool"] = test["Cabin"].notnull().astype('int')
sns.barplot(x="CabinBool", y="Survived", data=train,ax=ax[0])

# Строим каунтплот чтобы проверить теорию о том, что данные о каютах отсутствуют у более бедного класса
sns.countplot(x="Pclass", data=train[train.CabinBool == 0], ax=ax[1])
plt.show()

* Похоже, теория о том, что люди, у которых данные о каютах отсутствуют, были из более бедного класса, подтвердилась. Большая часть людей без номера каюты была из 2-го и 3-го класса.
* Тут же отмечу, что у пассажиров, у которых номер каюты указан, было больше шансов выжить (около 65%).
* В итоге решаю что **колонку Cabin можно исключить из датафреймов**, но созданную CabinBool оставлю.


In [ ]:
train = train.drop(['Cabin'], axis = 1)
test = test.drop(['Cabin'], axis = 1)

**Заполняем все оставшиеся пробелы**


Помним, что в **train** у нас еще осталась пара пропущенных значений в колонке **Embarked**, а в **test** есть пропущенное значение в **Fare**.

In [ ]:
# Смотрим, в каком порту больше всего село людей
sns.countplot(x="Embarked", data=train)
plt.show()

In [ ]:
# Раз большинство людей сели в Southampton (S), то им же пропущенные значения и заполним.
train = train.fillna({"Embarked": "S"})

In [ ]:
# Смотрим у кого не хватает данных о стоимости билета
fare_check = test[test['Fare'].isnull()]
fare_check

In [ ]:
# Вставляем вместо пропуска среднюю стоимость билета того же класса (3-го)
mean_fare = round(test[test.Pclass == 3].Fare.mean(), 4)
test = test.fillna({"Fare": mean_fare})

Проверяем, что все данные мы заполнили и идем дальше)

In [ ]:
train.isna().sum()

In [ ]:
test.isna().sum()

# 3. Визуализируем данные и смотрим у кого было больше шансов выжить

In [ ]:
fig, ax = plt.subplots(1, 6,figsize=(30,5))
sns.barplot(x="Pclass", y="Survived", data=train, ax=ax[0]).set_title('Class')
sns.barplot(x="Sex", y="Survived", data=train, ax=ax[1]).set_title('Sex')
sns.barplot(x="SibSp", y="Survived", data=train, ax=ax[2]).set_title('# of siblings / spouses aboard')
sns.barplot(x="Parch", y="Survived", data=train, ax=ax[3]).set_title('# of parents / children aboard')
sns.barplot(x="Embarked", y="Survived", data=train, ax=ax[4]).set_title('Port of Embarkation')
sns.countplot(x="Survived", data=train, ax=ax[5]).set_title('# of people survived / died')
plt.show()

**Наблюдения:**
* **Pclass** - Люди из более высокого класса имеют более высокую выживаемость (около 63%).
* **Sex** - Женщины имели гораздо больший шанс (около 74%) на выживание, чем мужчины.
* **SibSp** - Чем больше у тебя родственников, тем меньше шанс на выживание. Низкий процент у людей без родственников, скорее всего, объясняется тем, что в одиночку по большей части путешествовали мужчины.
* **Parch** - Люди, у которых менее четырех/пяти детей на борту выживут с большей вероятностью. Как и в предыдущем случае,  у людей, путешествующих в одиночку, меньше шансов выжить, чем у людей с 1–3 детьми. 
* **Embarked** - Как ни странно, но похоже что у людей севших в Cherbourg (C) шанс выжить (около 55%) был больше чем у людей севших в двух других портах.
* Большая часть людей погибла, выжили лишь около 39% людей.

**Отдельно смотрим Age, так как чтобы увидеть там что-то внятное надо сначала преобразовать данные.**

In [ ]:
plt.figure(figsize=(20,5))

# Делим возраст (континуальная переменная) на категории
bins = [-0.5, 5, 12, 18, 24, 35, 60, np.inf]
labels = ['Baby', 'Child', 'Teenager', 'Student', 'Young Adult', 'Adult', 'Senior']
train['Age_Group'] = pd.cut(train["Age"], bins, labels = labels)
test['Age_Group'] = pd.cut(test["Age"], bins, labels = labels)

# Рисуем график выживаемости по возрастным группам
sns.barplot(x="Age_Group", y="Survived", data=train)
plt.show()

Дети в возрасте до 5 лет имели наибольший шанс выжить, что ожидаемо, учитывая приказ [«Сначала женщины и дети»](https://en.wikipedia.org/wiki/Women_and_children_first).

# 4. Удаление ненужных данных
Заполнив пробелы и изучив данные, смотрим что у нас осталось и удаляем колонки, которые нам не понадобятся в будущем.

In [ ]:
train.head(3)

In [ ]:
test.head(3)

In [ ]:
# Данные о номере билета вряд ли коррелируют с выживаемостью, поэтому удаляем.
train = train.drop(['Ticket'], axis = 1)
test = test.drop(['Ticket'], axis = 1)

# ID пассажира в train тоже вряд ли коррелирует с выживаемостью, поэтому удаляем.
# В test оставляем, так как ID'шники потом понадобятся нам в итогом файле который будем сабмитить.
train = train.drop(['PassengerId'], axis = 1)

# 5. Создание новых признаков
Поскольку пока мои знания о ML оставляют желать лучшего, пришлось обратиться к [решению более опытного аналитика](https://www.kaggle.com/startupsci/titanic-data-science-solutions), чтобы узнать что делать дальше.

Из вышеупомянутого решения понял, что для обучения модели и построения хорошего прогноза необходимо понимать тип проблемы и требования к ее решению.
В нашем случае это **проблема классификации и регрессии**, так как мы хотим определить взаимосвязь между выходными данными (выжил человек или нет) с другими переменными / признаками (пол, возраст, порт и т.д). Помимо этого, стоит упомянуть, что категория машинного обучения, реализуемая здесь, называется **"Обучение с учителем"**, поскольку мы обучаем нашу модель с заданным набором данных. Используя эти два критерия - контролируемое обучение плюс классификация и регрессия, мы можем выбрать наиболее подходящиее для этого модели (список опять же взят [здесь](https://www.kaggle.com/startupsci/titanic-data-science-solutions)):
* Logistic Regression
* KNN or k-Nearest Neighbors
* Support Vector Machines
* Naive Bayes classifier
* Decision Tree
* Random Forrest
* Perceptron

Однако, прежде чем обучать модели, необходимо преобразовать оставшиеся параметры в числовые значения, т.к. этого требует большинство алгоритмов моделей. 
Также, помимо работы с существующими параметрами, в сопроводительном ролике к соревнованию, [советуют](https://www.youtube.com/watch?v=8yZMXCaFshs) поэксперементировать и попробовать задезайнить / создать свои параметры.

Код для создания переменной Title был взят [здесь](https://www.kaggle.com/startupsci/titanic-data-science-solutions).

In [ ]:
# Создаем объединенную группу из обоих датасетов
combine = [train, test]

# Достаем гоноратив из имен пассажиров в обоих датасетах и помещаем в отдельную колонку
for dataset in combine:
    dataset['Title'] = dataset.Name.str.extract(' ([A-Za-z]+)\.', expand=False)

# Смотрим что вытащили и как оно распределилось в зависиости от пола
pd.crosstab(train['Title'], train['Sex'])

In [ ]:
# Объединяем гоноративы в более крупные группы и смотрим у кого был больше шанс выжить
for dataset in combine:
    dataset['Title'] = dataset['Title'].replace(['Lady', 'Countess','Capt', 'Col','Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
    dataset['Title'] = dataset['Title'].replace('Mlle', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Ms', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Mme', 'Mrs')
    
train[['Title', 'Survived']].groupby(['Title'], as_index=False)\
                            .mean()\
                            .round(2)\
                            .sort_values(by='Survived', ascending=False)

In [ ]:
# Конвертируем текст в числа
title_map = {"Mrs": 1, "Miss": 2, "Master": 3, "Rare": 4, "Mr": 5}
for dataset in combine:
    dataset['Title'] = dataset['Title'].map(title_map)
    dataset['Title'] = dataset['Title'].fillna(0)

# Удаляем параметр 'Имя', т.к. дальше он нам уже не понадобится.
train = train.drop(['Name'], axis = 1)
test = test.drop(['Name'], axis = 1)

# Смотрим, что получилось
train.head(3)

In [ ]:
test.head(3)

# 6. Преобразоввание оставшихся признаков в числовые значения

**Признак Age_Group**

In [ ]:
# Конвертируем возрастные группы из текста в числа
age_map = {'Baby': 1, 'Child': 2, 'Teenager': 3, 'Student': 4, 'Young Adult': 5, 'Adult': 6, 'Senior': 7}
train['Age_Group'] = train['Age_Group'].map(age_map)
test['Age_Group'] = test['Age_Group'].map(age_map)

# Удаляем параметр 'Возраст', т.к. дальше он нам уже не понадобится.
train = train.drop(['Age'], axis = 1)
test = test.drop(['Age'], axis = 1)

**Признак Sex**

In [ ]:
# Конвертируем пол в бинарную переменную
sex_map = {"male": 0, "female": 1}
train['Sex'] = train['Sex'].map(sex_map)
test['Sex'] = test['Sex'].map(sex_map)

**Признак Embarked**

In [ ]:
# Конвертируем порты в числовые значения
embarked_map = {"S": 1, "C": 2, "Q": 3}
train['Embarked'] = train['Embarked'].map(embarked_map)
test['Embarked'] = test['Embarked'].map(embarked_map)

**Признак Fare**


Объединим в группы по тому же принципу что и возраст. Определение интервалов отдадим pandas'у. 

In [ ]:
train['Fare_Group'] = pd.cut(train["Fare"], 4)
train[['Fare_Group','Survived']].groupby(['Fare_Group'], as_index=False)\
                                .mean()\
                                .sort_values(by='Fare_Group', ascending=True)\
                                .round(2)

In [ ]:
# Присваиваем определенным выше интервалам номера от 1 до 4
bins = [-1, 128.082, 256.165, 384.247, 512.330]
labels = [1, 2, 3, 4]
train['Fare_Group'] = pd.cut(train["Fare"], bins, labels = labels)
test['Fare_Group'] = pd.cut(test["Fare"], bins, labels = labels)

# Удаляем параметр 'Fare', т.к. дальше он нам уже не понадобится.
train = train.drop(['Fare'], axis = 1)
test = test.drop(['Fare'], axis = 1)

**Смотрим еще раз как выглядят данные**

In [ ]:
train.head(3)

In [ ]:
test.head(3)

# 7. Тестируем и выбираем модель
Список моделей с которыми мы определились ранее:
* Logistic Regression
* KNN or k-Nearest Neighbors
* Support Vector Machines
* Naive Bayes
* Decision Tree
* Random Forrest
* Perceptron

Датасет train мы разделим на две части (80% и 20%) и будем использовать меньшую, чтобы проверять точность моделей. 

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

predictors = train.drop("Survived", axis=1)
target = train["Survived"]
x_train, x_val, y_train, y_val = train_test_split(predictors, target, test_size = 0.20, random_state = 0)

In [ ]:
# Logistic Regression
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression()
logreg.fit(x_train, y_train)
y_pred = logreg.predict(x_val)
acc_logreg = round(accuracy_score(y_pred, y_val) * 100, 2)
print(acc_logreg)

In [ ]:
# KNN or k-Nearest Neighbors
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
knn.fit(x_train, y_train)
y_pred = knn.predict(x_val)
acc_knn = round(accuracy_score(y_pred, y_val) * 100, 2)
print(acc_knn)

In [ ]:
# Support Vector Machines
from sklearn.svm import SVC

svc = SVC()
svc.fit(x_train, y_train)
y_pred = svc.predict(x_val)
acc_svc = round(accuracy_score(y_pred, y_val) * 100, 2)
print(acc_svc)

In [ ]:
# Gaussian Naive Bayes
from sklearn.naive_bayes import GaussianNB

gaussian = GaussianNB()
gaussian.fit(x_train, y_train)
y_pred = gaussian.predict(x_val)
acc_gaussian = round(accuracy_score(y_pred, y_val) * 100, 2)
print(acc_gaussian)

In [ ]:
# Decision Tree
from sklearn.tree import DecisionTreeClassifier

decisiontree = DecisionTreeClassifier()
decisiontree.fit(x_train, y_train)
y_pred = decisiontree.predict(x_val)
acc_decisiontree = round(accuracy_score(y_pred, y_val) * 100, 2)
print(acc_decisiontree)

In [ ]:
# Random Forest
from sklearn.ensemble import RandomForestClassifier

randomforest = RandomForestClassifier()
randomforest.fit(x_train, y_train)
y_pred = randomforest.predict(x_val)
acc_randomforest = round(accuracy_score(y_pred, y_val) * 100, 2)
print(acc_randomforest)

In [ ]:
# Perceptron
from sklearn.linear_model import Perceptron

perceptron = Perceptron()
perceptron.fit(x_train, y_train)
y_pred = perceptron.predict(x_val)
acc_perceptron = round(accuracy_score(y_pred, y_val) * 100, 2)
print(acc_perceptron)

Сравнимаем результаты

In [ ]:
models = pd.DataFrame({
    'Model': ['Logistic Regression', 'KNN', 'Support Vector Machines', 'Gaussian Naive Bayes', 'Decision Tree', 'Random Forest', 'Perceptron'],
    'Score': [acc_logreg, acc_knn, acc_svc, acc_gaussian, acc_decisiontree, acc_randomforest, acc_perceptron]})
models.sort_values(by='Score', ascending=False)

# 8. Загружаем лучший результат

In [ ]:
# Cоздаем переменные с ID пассажиров из test и предсказанной выживаемостью
ID = test['PassengerId']
predictions = gaussian.predict(test.drop('PassengerId', axis=1))

# В датафрейм output помещаем данные из созданных выше переменных и сохраняем в csv-файл
output = pd.DataFrame({'PassengerId' : ID, 'Survived': predictions})
output.to_csv('my_submission.csv', index=False)
print("Your submission was successfully saved!")

**Источники, изучение которых помогло лучше разобраться что как работает**
* https://www.kaggle.com/startupsci/titanic-data-science-solutions
* https://www.kaggle.com/nadintamer/titanic-survival-predictions-beginner/comments